In [16]:
import os
import json
import numpy as np
from scipy.spatial.transform import Rotation

In [17]:
SAVE_PATH = os.path.join(os.getcwd(), '../data/one_spin')
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
if not os.path.exists(os.path.join(SAVE_PATH, 'images')):
    os.makedirs(os.path.join(SAVE_PATH, 'images'))

In [18]:
transforms = {}
transforms["fl_x"] = 762.72 # focal length in x
transforms["fl_y"] = 762.72# focal length in y
transforms["w"] = 1280 # image width
transforms["h"] = 720 # image height
transforms["cx"] = transforms["w"]//2 # optical center in x
transforms["cy"] = transforms["h"]//2 # optical center in y
transforms["k1"] = 0
transforms["k2"] = 0
transforms["k3"] = 0
transforms["p1"] = 0
transforms["p2"] = 0
transforms["camera_model"] = "OPENCV" # camera model

In [19]:
frames = []

starting_pts = [[15, 0]]
z = 20
pitch = 0.7
spin = 20

cnt = 0

for x, y in starting_pts:
    start_transform_matrix = np.eye(4)
    start_transform_matrix[0, 3] = x
    start_transform_matrix[1, 3] = y
    start_transform_matrix[2, 3] = z

    #Pitch down cammera
    pitch_matrix = np.eye(4)
    pitch_matrix[1, 1] = np.cos(pitch)
    pitch_matrix[1, 2] = -np.sin(pitch)
    pitch_matrix[2, 1] = np.sin(pitch)
    pitch_matrix[2, 2] = np.cos(pitch)

    start_transform_matrix = np.matmul(pitch_matrix, start_transform_matrix)

    for i in range(spin):
        angle = i * 2 * np.pi / spin
        rot = Rotation.from_rotvec(np.array([0, 0, 1]) * angle).as_matrix()
        rot_matrix = np.eye(4)
        rot_matrix[:3, :3] = rot
        transform_matrix = np.matmul(rot_matrix, start_transform_matrix)

        frame = {}
        file_path = "images/{}.png".format(cnt)

        frame["file_path"] = file_path
        frame["transform_matrix"] = transform_matrix.tolist()
        print(transform_matrix.tolist())

        frames.append(frame)

        cnt += 1


transforms["frames"] = frames

[[1.0, 0.0, 0.0, 15.0], [0.0, 0.7648421872844885, -0.644217687237691, -12.88435374475382], [0.0, 0.644217687237691, 0.7648421872844885, 15.29684374568977], [0.0, 0.0, 0.0, 1.0]]
[[0.9510565162951536, -0.23634923388581328, 0.19907421343337123, 18.24733201309473], [0.30901699437494745, 0.7274081461543511, -0.6126874293599993, -7.618493671575774], [0.0, 0.644217687237691, 0.7648421872844885, 15.29684374568977], [0.0, 0.0, 0.0, 1.0]]
[[0.8090169943749473, -0.44956295801693996, 0.3786616558242797, 19.708488032109805], [0.587785252292473, 0.6187703275280575, -0.5211830570522167, -1.606882356657238], [0.0, 0.6442176872376909, 0.7648421872844884, 15.296843745689769], [0.0, 0.0, 0.0, 1.0]]
[[0.5877852522924732, -0.6187703275280575, 0.5211830570522167, 19.24043992543143], [0.8090169943749473, 0.44956295801694013, -0.37866165582427985, 4.5620217991386145], [0.0, 0.644217687237691, 0.7648421872844885, 15.29684374568977], [0.0, 0.0, 0.0, 1.0]]
[[0.30901699437494745, -0.7274081461543511, 0.612687429

In [20]:
#Save transforms
with open(os.path.join(SAVE_PATH,'transforms.json'), 'w') as outfile:
    json.dump(transforms, outfile)